In [1]:
#Authors Amulya J N  ojaswini2004@gmail.com , Nikitha M N  nikitha.2723@gmail.com

import pandas as pd
import folium
from folium.plugins import AntPath
from geopy.distance import distance
import pygame
from twilio.rest import Client
import tkinter as tk
from tkinter import messagebox
from tkinter import ttk
import webbrowser

# Initialize pygame mixer
pygame.mixer.init()

# Twilio Account SID and Auth Token
account_sid = 'ACf9f557301feeae5ff745601d8fd2ae51'
auth_token = '956c1ba96cca3d6e8b8123c1842ce0e5'

# Configure Twilio client
twilio_client = Client(account_sid, auth_token)

# Function to check if points overlap
def check_overlap(df1, df2):
    points1 = [(lat, lon) for lat, lon in zip(df1['Latitude'], df1['Longitude'])]
    points2 = [(lat, lon) for lat, lon in zip(df2['Latitude'], df2['Longitude'])]

    for point1 in points1:
        for point2 in points2:
            if distance(point1, point2).meters <= 10:
                return True
    return False

# Read CSV files
file1 = 'predictions.csv'
file2 = 'Village_outline.csv'

df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)

# Create folium map for file1 animation
mapObj = folium.Map(location=[11.85442,76.39554], zoom_start=15)

# Extract latitude and longitude columns for file1 and file2
pathLatLngs1 = df1[['Latitude', 'Longitude']].values.tolist()
pathLatLngs2 = df2[['Latitude', 'Longitude']].values.tolist()

# Define colors for each dataset
color1 = "blue"
color2 = "red"

# Add AntPath with animation for file1 with blue color and file2 with red color (static path)
ant_path1 = AntPath(pathLatLngs1, delay=400, weight=3, color=color1, dash_array=[30, 15]).add_to(mapObj)
folium.PolyLine(pathLatLngs2, weight=3, color=color2).add_to(mapObj)  # Static path

# Save the map before any possible modifications
mapObj.save("output_before.html")

def check_and_notify():
    if check_overlap(df1, df2):
        print("Overlapping points found! Reversing animation direction for Prediction.")
        reversed_pathLatLngs1 = pathLatLngs1[::-1]  # Reverse the path
        ant_path1.locations = reversed_pathLatLngs1  # Update the AntPath object
        pygame.mixer.music.load('sound.mp3')  #  path to  sound file
        pygame.mixer.music.play()
        
        # Send SMS notification using Twilio
        from_number = '+15856362995'  # Twilio number
        to_number = '+6360039654'   # Recipient's phone number
        message = 'Predicted location within village area!'

        try:
            message = twilio_client.messages.create(
                body=message,
                from_=from_number,
                to=to_number
            )
            print(f"SMS sent successfully! SID: {message.sid}")
            messagebox.showinfo("Notification", "Overlapping points found! SMS sent successfully!")
        except Exception as e:
            print(f"Error sending SMS: {str(e)}")
            messagebox.showerror("Error", f"Error sending SMS: {str(e)}")
    else:
        print("No overlapping points found.")
        messagebox.showinfo("Notification", "No overlapping points found.")
    
    # Save the updated map
    mapObj.save("output_after.html")

def open_map_before():
    webbrowser.open("output_before.html")

def open_map_after():
    webbrowser.open("output_after.html")

# Create GUI application
root = tk.Tk()
root.title("Overlap Checker")

frame = ttk.Frame(root, padding="10")
frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))

# Add Check button
check_button = ttk.Button(frame, text="Check Overlap", command=check_and_notify)
check_button.grid(row=0, column=0, padx=5, pady=5)

# Add buttons to view map before and after
view_before_button = ttk.Button(frame, text="View Map Before", command=open_map_before)
view_before_button.grid(row=1, column=0, padx=5, pady=5)

view_after_button = ttk.Button(frame, text="View Map After", command=open_map_after)
view_after_button.grid(row=2, column=0, padx=5, pady=5)

root.mainloop()


pygame 2.6.0 (SDL 2.28.4, Python 3.12.3)
Hello from the pygame community. https://www.pygame.org/contribute.html
Overlapping points found! Reversing animation direction for Prediction.
Error sending SMS: HTTP 400 error: Unable to create record: Invalid 'To' Phone Number: +636003XXXX
